In [1]:
using ReactiveMP
using StaticArrays
using BenchmarkTools
using Distributions
using Rocket

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1260
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [2]:
node = GaussianMeanVariance()

Node{Normal{Float64},3,SArray{Tuple{1},Array{Int64,1},1,1}}(NodeVariable[mean, variance, value], [[1, 2, 3]])

In [3]:
mean     = constvar(:mean, 1.0)
variance = constvar(:var, 0.0) 

ConstVariable{SingleObservable{Message{Float64},AsapScheduler}}(:var, SingleObservable(Message{Float64}, AsapScheduler), LazyObservable(ReactiveMP.AbstractMessage))

In [4]:
connect!(node, :mean, mean, 1)
connect!(node, :variance, variance, 1)

In [5]:
activate!(node)

In [6]:
valueindex = varindex(node, :value)
valuevar   = @inbounds variables(node)[valueindex]

value

In [7]:
subscribe!(messageout(valuevar), logger())

[LogActor] Data: Message{Normal{Float64}}(Normal{Float64}(μ=1.0, σ=0.0))
[LogActor] Completed


SwitchMapSubscription()

In [8]:
addition = AdditionNode()

Node{typeof(+),3,SArray{Tuple{1},SArray{Tuple{3},Int64,1,3},1,1}}(NodeVariable[in1, in2, out], SArray{Tuple{3},Int64,1,3}[[1, 2, 3]])

In [9]:
connect!(addition, :in1, mean, 1)
connect!(addition, :in2, variance, 1)

In [10]:
activate!(addition)

In [11]:
subscribe!(messageout(getvariable(addition, :out)), logger())

[LogActor] Error: MethodError(ReactiveMP.rule, (typeof(+), Val{:out}(), (Message{Float64}(1.0), Message{Float64}(0.0)), nothing, nothing), 0x0000000000006a24)


SwitchMapSubscription()

In [79]:
functionalform(addition)

typeof(+)

In [13]:
rule(functionalform(addition), Val(:out), (Message(0.0), Message(1.0)), nothing, nothing)

MethodError: MethodError: no method matching rule(::Type{typeof(+)}, ::Val{:out}, ::Tuple{Message{Float64},Message{Float64}}, ::Nothing, ::Nothing)
Closest candidates are:
  rule(!Matched::typeof(+), ::Val{:out}, ::Tuple{Message{T},Message{T}}, ::Nothing, ::Nothing) where T at /Users/bvdmitri/.julia/dev/ReactiveMP/src/nodes/addition.jl:10
  rule(!Matched::Type{#s20} where #s20<:Normal{T}, !Matched::Val{:value}, ::Tuple{Message{T},Message{T}}, ::Nothing, ::Nothing) where T at /Users/bvdmitri/.julia/dev/ReactiveMP/src/nodes/gaussian.jl:9
  rule(!Matched::typeof(+), ::Val{:out}, !Matched::Tuple{Message{Normal{T}},Message{Normal{T}}}, ::Nothing, ::Nothing) where T at /Users/bvdmitri/.julia/dev/ReactiveMP/src/nodes/addition.jl:14
  ...